In [1]:
import pandas as pd

In [2]:
!gcloud

zsh:1: command not found: gcloud


In [ ]:
# pip install google-cloud-storage
# pip install pandas-gbq

In [5]:

from google.cloud import storage
from google.oauth2 import service_account
import json 
 
 # si ce code est ouvert dans cloud shell, plus besoin de fichier credentials
with open("ensai-2024-37c511689d57.json") as source:
    info = json.load(source)


credentials = service_account.Credentials.from_service_account_info(info)
prenoms = pd.read_gbq("ml.prenoms", project_id="ensai-2024", credentials=credentials)

In [6]:
prenoms.head()

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,13,37
1,1,_PRENOMS_RARES,1900,57,37
2,1,_PRENOMS_RARES,1900,59,44
3,1,_PRENOMS_RARES,1900,62,34
4,1,_PRENOMS_RARES,1900,75,45


### Models

In [8]:
len(set(prenoms["preusuel"]))

36171

### Preprocessing

In [36]:
import sklearn 
from sklearn.model_selection import train_test_split

preprocessing = prenoms[prenoms["preusuel"] != '_PRENOMS_RARES']
preprocessing["nombre"] = preprocessing["nombre"].astype(int)
preprocessing = preprocessing.groupby(["preusuel", "sexe"]).agg({"nombre": sum}).reset_index()

preprocessing

/var/folders/fc/1rmw025j2fg0q_wyg876ltcm0000gn/T/ipykernel_59681/3475075879.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessing["nombre"] = preprocessing["nombre"].astype(int)


,preusuel,sexe,nombre
0,A,1,27
1,AADAM,1,32
2,AADEL,1,57
3,AADHIRA,2,22
4,AADIL,1,190
...,...,...,...
38472,ÖZGE,2,31
38473,ÖZGÜR,1,26
38474,ÖZKAN,1,27
38475,ÖZLEM,2,108


In [32]:
data = (
    preprocessing
        .iloc[preprocessing.groupby("preusuel")["nombre"].idxmax()]
        .drop(["nombre"], axis=1)
)

### Feature Engineering

In [40]:
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

In [45]:
import string
def encode_prenom(prenom: str):
    '''
    
    '''
    alphabet = string.ascii_lowercase + 'é-'
    prenom = prenom.lower()
    
    return pd.Series([letter in prenom for letter in alphabet]).astype(int)

X = data["preusuel"].apply(encode_prenom)

In [46]:
X
Y = (data["sexe"].astype(int) - 1)

In [47]:
X
Y = (data["sexe"].astype(int) - 1)
X.head(), Y.head()

(   0   1   2   3   4   5   6   7   8   9   ...  18  19  20  21  22  23  24  \
 0   1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 1   1   0   0   1   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 2   1   0   0   1   1   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 3   1   0   0   1   0   0   0   1   1   0  ...   0   0   0   0   0   0   0   
 4   1   0   0   1   0   0   0   0   1   0  ...   0   0   0   0   0   0   0   
 
    25  26  27  
 0   0   0   0  
 1   0   0   0  
 2   0   0   0  
 3   0   0   0  
 4   0   0   0  
 
 [5 rows x 28 columns],
 0    0
 1    0
 2    0
 3    1
 4    0
 Name: sexe, dtype: int64)

### Modelisation

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
regr  = LogisticRegression()
regr.fit(X, Y)


LogisticRegression()

In [57]:
regr.predict([encode_prenom('Alain')])

array([1])

In [58]:
y_pred = regr.predict(X)
accuracy_score(Y, y_pred)

0.6458114459496821

# Dump du modele

In [59]:
import joblib
joblib.dump(regr, "model.v1.bin")

['model.v1.bin']

# Reload du modele 

In [61]:
regr_loaded = joblib.load("model.v1.bin")
regr_loaded.predict([encode_prenom("Toto")])


array([0])